# Running on Preimplantant embryos EMATB3929

Here we tested the dataset available at accession number: [E-MATB-3929](https://www.ebi.ac.uk/arrayexpress/experiments/E-MTAB-3929/). Link to Paper: [Single-cell RNA-seq reveal lineage formation and X-chromosome dosage compensation in human preimplantation embryos](https://doi.org/10.1016/j.stemcr.2013.10.009)

<div id="toc"></div>

## Neccessary Imports

In [19]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [20]:
import sys
code = "./../../code/"
data = "./../../data/"
sys.path.append(code)
import pandas
import pypairs as pairs
from sklearn.preprocessing import QuantileTransformer
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np
import colorlover as cl
from pathlib import Path
from tqdm import tqdm_notebook as tqdm
import helper
import timeit

init_notebook_mode(connected=True)

In [21]:
cc_marker = helper.load_ocope_marker(data, fraction=0.6)

[__set_matrix] Original Matrix 'x' has shape 19084 x 247
[__set_matrix] Removed 16689 genes that were not in 'subset_genes'. 2395 genes remaining.
[__set_matrix] Removed 61 genes that were not expressed in any samples. 2334 genes remaining.
[__set_matrix] Removed 0 samples that were not annotated in 'phases'. 247 samples remaining.
[__set_matrix] Matrix truncation done. Working with 2334 genes for 247 samples.
[sandbag] Identifying marker pairs...Processing in parallel with 10 processes...
 Done!
[sandbag] Identified 8146 marker pairs (phase: count): {'G1': 2575, 'S': 4101, 'G2M': 1470}


## Human preimplantation embryos
Human parthenogenetic ES from [Single-Cell RNA-Seq Reveals Lineage and X Chromosome Dynamics in Human Preimplantation Embryos](http://www.cell.com/cell/fulltext/S0092-8674(16)30280-X?_returnURL=https%3A%2F%2Flinkinghub.elsevier.com%2Fretrieve%2Fpii%2FS009286741630280X%3Fshowall%3Dtrue)

## Counts

In [22]:
gencounts_EMTAB3929_counts = pandas.read_csv(Path(data + "E-MTAB-3929.processed.1_counts.txt"), sep='\t')

gencounts_EMTAB3929_counts.set_index("Unnamed: 0", inplace=True)
gencounts_EMTAB3929_counts.head(10)

,E5.5.101,E5.5.100,E6.2.114,E6.2.104,E6.2.107,E6.2.116,E7.2.138,E6.2.118,E6.2.105,E7.2.144,...,E3.50.3415,E3.51.3421,E3.53.3437,E3.51.3423,E3.52.3429,E3.49.3407,E3.51.3426,E3.47.3391,E3.52.3431,E3.53.3438
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A1BG,0,0,0,0,0,16,0,0,0,0,...,327,2167,170,451,104,446,2517,473,104,116
A1BG-AS1,0,0,0,0,0,0,0,0,0,0,...,0,88,0,0,0,0,1,7,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,39,86,0,0,15,11,94,40,66,34
A2M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M-AS1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2ML1,3,0,0,0,0,0,1,0,0,0,...,26,158,0,0,137,3,0,0,124,10
A2MP1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A3GALT2,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
A4GALT,50,123,20,18,54,291,111,349,232,343,...,44,120,33,12,68,0,96,0,12,13


## Predicting

In [23]:
x = gencounts_EMTAB3929_counts.T.values

X_std = QuantileTransformer().fit_transform(x.astype(float))

gencounts_EMTAB3929_counts_Qnorm = pandas.DataFrame(X_std.T, index=gencounts_EMTAB3929_counts.index, columns=gencounts_EMTAB3929_counts.columns)

EMTAB3929_counts_prediction = pairs.cyclone(gencounts_EMTAB3929_counts, cc_marker, verbose=True, processes=0)

EMTAB3929_counts_prediction_table = helper.get_prediction_table(EMTAB3929_counts_prediction)
helper.DataTable(EMTAB3929_counts_prediction_table)

[__set_matrix] Original Matrix 'x' has shape 26178 x 1529
[__set_matrix] Matrix truncation done. Working with 26178 genes for 1529 samples.
[cyclone] Preparing marker pairs, where at least one gene was not present in 'x'... Done!
[cyclone] Removed 229 marker pairs. 8146 marker pairs remaining.
[cyclone] Calculating scores and predicting cell cycle phase... Done!
[cyclone] Calculated scores and prediction (phase: count): G2M: 955, G1: 199, S: 375


,G1,G2M,S,G1_norm,G2M_norm,S_norm,prediction
sample,,,,,,,
E5.5.101,0.169,1.000,0.000,0.144568,0.855432,0.000000,G2M
E5.5.100,0.024,0.993,0.000,0.023599,0.976401,0.000000,G2M
E6.2.114,0.013,0.959,0.280,0.010383,0.765974,0.223642,G2M
E6.2.104,0.635,0.004,0.095,0.865123,0.005450,0.129428,G1
E6.2.107,0.057,0.907,0.100,0.053571,0.852444,0.093985,G2M
E6.2.116,0.174,0.781,0.005,0.181250,0.813542,0.005208,G2M
E7.2.138,0.458,0.999,0.000,0.314345,0.685655,0.000000,G2M
E6.2.118,0.076,0.572,0.716,0.055718,0.419355,0.524927,G2M
E6.2.105,0.202,0.670,0.053,0.218378,0.724324,0.057297,G2M


## Plotting prediction

In [24]:
classes = ["E3","E4.late","E4","E5.early","E5","E6","E7"]
labels = []
for idx in EMTAB3929_counts_prediction_table.index:
    for c in classes:
        if c in idx:
           labels.append(c) 
           break

In [25]:
e3_g1 = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E3"],0].values
e3_g2m = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E3"],1].values
print("E3 {}".format(len(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E3"],1].values)))

e4_g1 = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E4"],0].values
e4_g2m = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E4"],1].values
print("E4 {}".format(len(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E4"],1].values)))


e4late_g1 = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E4.late"],0].values
e4late_g2m = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E4.late"],1].values
print("E4 late {}".format(len(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E4.late"],1].values)))


e5early_g1 = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E5.early"],0].values
e5early_g2m = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E5.early"],1].values
print("E5 early {}".format(len(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E5.early"],1].values)))

e5_g1 = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E5"],0].values
e5_g2m = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E5"],1].values
print("E5 {}".format(len(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E5"],1].values)))

e6_g1 = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E6"],0].values
e6_g2m = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E6"],1].values
print("E6 {}".format(len(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E6"],1].values)))

e7_g1 = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E7"],0].values
e7_g2m = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E7"],1].values
print("E7 {}".format(len(EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E7"],1].values)))

E3 81
E4 142
E4 late 48
E5 early 24
E5 353
E6 415
E7 466


In [26]:
set1 = cl.scales['9']['qual']['Set1']

e3_trace = go.Scatter(
    x = e3_g1,
    y = e3_g2m,
    mode='markers+text',
    marker=dict(
        symbol='circle',
        size=10,
        color=set1[1],
    ),
    name='E3'
)

e4_trace = go.Scatter(
    x = e4_g1,
    y = e4_g2m,
    mode='markers+text',
    marker=dict(
        symbol='circle',
        size=10,
        color=set1[2],
    ),
    name='E4'
)

e4late_trace = go.Scatter(
    x = e4late_g1,
    y = e4late_g2m,
    mode='markers+text',
    marker=dict(
        symbol='circle',
        size=10,
        color=set1[3],
    ),
    name='E4 late'
)


e5early_trace = go.Scatter(
    x = e5early_g1,
    y = e5early_g2m,
    mode='markers+text',
    marker=dict(
        symbol='circle',
        size=10,
        color=set1[4],
    ),
    name='E5 early'
)

e5_trace = go.Scatter(
    x = e5_g1,
    y = e5_g2m,
    mode='markers+text',
    marker=dict(
        symbol='circle',
        size=10,
        color=set1[5],
    ),
    name='E5'
)

e6_trace = go.Scatter(
    x = e6_g1,
    y = e6_g2m,
    mode='markers+text',
    marker=dict(
        symbol='circle',
        size=10,
        color=set1[6],
    ),
    name='E6'
)


e7_trace = go.Scatter(
    x = e7_g1,
    y = e7_g2m,
    mode='markers+text',
    marker=dict(
        symbol='circle',
        size=10,
        color=set1[7],
    ),
    name='E7'
)


lbls = go.Scatter(
    x=[0.8, 0.4, 0.25],
    y=[0.4, 0.8, 0.25],
    text=['G1',
          'G2M',
          'S'],
    mode='text',
    showlegend=False,
    hoverinfo='none'
)

data = [e3_trace, e4_trace, e4late_trace, e5early_trace, lbls]
#e5_trace, e6_trace, e7_trace,

layout = {
    'title': "Predicted cell cycle clustering for E3 - early E5",
    'xaxis': {
        'title': "G1 Score",
        'range': [-0.1, 1.1],
    },
    'yaxis': {
        'title': "G2M Score",
        'range': [-0.1, 1.1]
    },
    'shapes': [
        # G1
        {
            'type': 'path',
            'path': ' M 0.5,0 L1,0 L1,1 L0.5,0.5 Z',
            'fillcolor': 'rgba(255,0,0,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        # S
        {
            'type': 'path',
            'path': ' M 0,0 L0.5,0 L0.5,0.5 L0,0.5 Z',
            'fillcolor': 'rgba(255,255,0,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        # G2M
        {
            'type': 'path',
            'path': ' M 0,0.5 L0,1 L1,1 L0.5,0.5 Z',
            'fillcolor': 'rgba(0,0,255,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        {
            'type': 'circle',
            'xref': 'x',
            'yref': 'y',
            'x0': np.percentile(list(e3_g1) + list(e4_g1) + list(e4late_g1) + list(e5early_g1), 5),
            'y0': np.percentile(list(e3_g2m) + list(e4_g2m) + list(e4late_g2m) + list(e5early_g2m), 5),
            'x1': np.percentile(list(e3_g1) + list(e4_g1) + list(e4late_g1) + list(e5early_g1), 95),
            'y1': np.percentile(list(e3_g2m) + list(e4_g2m) + list(e4late_g2m) + list(e5early_g2m), 95),
            'opacity': 0.9,
            #'fillcolor': 'black',
            'line': {
                'color': 'red',
                'width': 5,
                'dash': 'dot'
            },
        },
        #{
        #    'type': 'circle',
        #    'xref': 'x',
        #    'yref': 'y',
        #    'x0': np.percentile(list(e5_g1) + list(e6_g1) + list(e7_g1), 5),
        #    'y0': np.percentile(list(e5_g2m) + list(e6_g2m) + list(e7_g2m), 5),
        #    'x1': np.percentile(list(e5_g1) + list(e6_g1) + list(e7_g1), 95),
        #    'y1': np.percentile(list(e5_g2m) + list(e6_g2m) + list(e7_g2m), 95),
        #    'opacity': 0.8,
        #    'fillcolor': 'blue',
        #    'line': {
        #        'color': 'black',
        #    },
        #},
    ]
}
fig = {
    'data': data,
    'layout': layout,
}

iplot(fig)

In [27]:
data = [e5_trace, e6_trace, e7_trace, lbls]
#

layout = {
    'title': "Predicted cell cycle clustering for E5 - E7",
    'xaxis': {
        'title': "G1 Score",
        'range': [-0.1, 1.1],
    },
    'yaxis': {
        'title': "G2M Score",
        'range': [-0.1, 1.1]
    },
    'shapes': [
        # G1
        {
            'type': 'path',
            'path': ' M 0.5,0 L1,0 L1,1 L0.5,0.5 Z',
            'fillcolor': 'rgba(255,0,0,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        # S
        {
            'type': 'path',
            'path': ' M 0,0 L0.5,0 L0.5,0.5 L0,0.5 Z',
            'fillcolor': 'rgba(255,255,0,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        # G2M
        {
            'type': 'path',
            'path': ' M 0,0.5 L0,1 L1,1 L0.5,0.5 Z',
            'fillcolor': 'rgba(0,0,255,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        {
            'type': 'circle',
            'xref': 'x',
            'yref': 'y',
            'x0': np.percentile(list(e5_g1) + list(e6_g1) + list(e7_g1), 5),
            'y0': np.percentile(list(e5_g2m) + list(e6_g2m) + list(e7_g2m), 5),
            'x1': np.percentile(list(e5_g1) + list(e6_g1) + list(e7_g1), 95),
            'y1': np.percentile(list(e5_g2m) + list(e6_g2m) + list(e7_g2m), 95),
            'opacity': 0.9,
            #'fillcolor': 'black',
            'line': {
                'color': 'red',
                'width': 5,
                'dash': 'dot'
            },
        },
    ]
}
fig = {
    'data': data,
    'layout': layout,
}

iplot(fig)

## Analyzing a single stage in depth

In [28]:
from collections import defaultdict

e3 = EMTAB3929_counts_prediction_table.iloc[[i for i, l in enumerate(labels) if l == "E6"],:]
sub = defaultdict(list)
for index, row in e3.iterrows():
    pos = len(index) - 1 - index[::-1].index(".")
    s = index[:pos]
    sub[s].append((row[0], row[1], row[6]))
    
sub

defaultdict(list,
            {'E6.1': [(0.171, 0.163, 'S'),
              (0.447, 0.177, 'S'),
              (0.15, 0.057, 'S'),
              (0.258, 0.954, 'G2M'),
              (0.019, 0.116, 'S'),
              (0.887, 0.08, 'G1'),
              (0.119, 0.084, 'S'),
              (0.081, 0.589, 'G2M'),
              (0.132, 0.828, 'G2M'),
              (0.036, 0.021, 'S'),
              (0.07, 1.0, 'G2M'),
              (0.071, 1.0, 'G2M'),
              (0.735, 0.0, 'G1'),
              (0.827, 0.02, 'G1'),
              (0.551, 0.004, 'G1'),
              (0.043, 0.266, 'S')],
             'E6.10': [(0.899, 0.073, 'G1'),
              (0.977, 0.14, 'G1'),
              (0.288, 0.018, 'S'),
              (0.33, 0.222, 'S'),
              (0.348, 0.204, 'S'),
              (0.644, 0.008, 'G1'),
              (0.507, 0.211, 'G1'),
              (0.967, 0.001, 'G1'),
              (0.026, 0.49, 'S'),
              (0.511, 0.0, 'G1'),
              (0.223, 0.0, 'S'),
              (0

In [29]:
import colorlover as cl


bupu = cl.scales['7']['qual']['Set1']
bupu500 = cl.interp( bupu, 20 )

traces = []
i = 0
for key, val in sub.items():
    trace = go.Scatter(
        x = [i[0] for i in val],
        y = [i[1] for i in val],
        mode='markers+text',
        marker=dict(
            symbol='circle',
            size=10,
            color=bupu500[i],
        ),
        name=key
    )
    i+=1
    traces.append(trace)

layout = {
    'xaxis': {
        'range': [-0.1, 1.1],
    },
    'yaxis': {
        'range': [-0.1, 1.1]
    },
    'shapes': [
        # G1
        {
            'type': 'path',
            'path': ' M 0.5,0 L1,0 L1,1 L0.5,0.5 Z',
            'fillcolor': 'rgba(255,0,0,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        # S
        {
            'type': 'path',
            'path': ' M 0,0 L0.5,0 L0.5,0.5 L0,0.5 Z',
            'fillcolor': 'rgba(255,255,0,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        },
        # G2M
        {
            'type': 'path',
            'path': ' M 0,0.5 L0,1 L1,1 L0.5,0.5 Z',
            'fillcolor': 'rgba(0,0,255,0.1)',
            'line': {
                'width': 1,
                'dash': 'dash'
            }
        }
    ]
}
fig = {
    'data': traces,
    'layout': layout,
}

iplot(fig)